In [ ]:
!pip install -q roboflow
!pip install -U tensorflow

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from roboflow import Roboflow


print("TensorFlow version:", tf.__version__)

# ============================
# 1. Download dataset from Roboflow
# ============================
API_KEY        = "*************"
WORKSPACE      = "ddd8889"
PROJECT        = "hand-gestures-2-4w8vc"
VERSION_NUMBER = 1

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
version = project.version(VERSION_NUMBER)

dataset = version.download("folder")       # classification export
dataset_dir = dataset.location

train_dir = os.path.join(dataset_dir, "train")
val_dir   = os.path.join(dataset_dir, "valid")
test_dir  = os.path.join(dataset_dir, "test")

# ============================
# 2. Build tf.data datasets
# ============================
IMG_SIZE    = (224, 224)
BATCH_SIZE  = 48
SEED        = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels="inferred",
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)
def make_sparse_label_smoothing_loss(num_classes, epsilon=0.1):
    def loss(y_true, y_pred):
        # y_true: (batch,) or (batch, 1) -> flatten to (batch,)
        y_true = tf.cast(tf.reshape(y_true, [-1]), tf.int32)

        # One-hot encode
        y_true_oh = tf.one_hot(y_true, depth=num_classes)

        # Apply label smoothing
        epsilon = tf.cast(epsilon, tf.float32)
        num_classes_f = tf.cast(num_classes, tf.float32)
        y_true_smooth = y_true_oh * (1.0 - epsilon) + epsilon / num_classes_f

        # Standard categorical cross-entropy on smoothed labels
        return tf.keras.losses.categorical_crossentropy(
            y_true_smooth, y_pred
        )
    return loss


AUTOTUNE = tf.data.AUTOTUNE
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# ============================
# 2.5 Data Augmentation + Preprocess
# ============================
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.25),
    layers.RandomZoom(0.25),
    layers.RandomContrast(0.3),
    layers.RandomBrightness(0.3),
    layers.GaussianNoise(0.05),
])



def augment(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
val_ds   = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
test_ds  = test_ds.map(preprocess, num_parallel_calls=AUTOTUNE)

train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)

# ============================
# 3. Build Customized MobileNetV2 Model
# ============================

# Wider MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet",
    alpha=1.3   #wider model (better accuracy)
)

base_model.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,))

# Data augmentation can also be applied here if preferred
x = inputs

# Feature extractor
x = base_model(x, training=False)

# Global pooling
x = layers.GlobalAveragePooling2D()(x)

# CUSTOM LAYERS
#still transfer-learning
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

# Output layer
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


# ============================
# 4. Train (Frozen base)
# ============================
EPOCHS = 20
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
)

# ============================
# 5. Fine-tuning
# ============================

# Unfreeze deeper 40% of layers
base_model.trainable = True

fine_tune_at = int(len(base_model.layers) * 0.6)

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

FINE_TUNE_EPOCHS = 15

history_ft = model.fit(
    train_ds,
    epochs=EPOCHS + FINE_TUNE_EPOCHS,
    initial_epoch=history.epoch[-1],
    validation_data=val_ds,
)


# ============================
# 6. Evaluate on test set
# ============================
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

# ============================
# 7. Save model
# ============================
model.save("mobilenetv2_roboflow_classification.h5")
model.save("mobilenetv2_roboflow_classification.keras")

print("Saved model successfully!")
print("Class names:", class_names)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.


Extracting Dataset Version Zip to hand-gestures-2-1 in folder:: 100%|██████████| 5254/5254 [00:00<00:00, 9669.85it/s]


Found 3664 files belonging to 5 classes.
Found 1047 files belonging to 5 classes.
Found 524 files belonging to 5 classes.
Classes: ['fist', 'one-finger-up', 'open-palm', 'thumps-up', 'two-fingers']
15441408/15441408 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.30_224            │ (None, 7, 7, 1664)     │     3,766,048 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1664)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1664)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       426,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,227,365 (16.13 MB)

 Trainable params: 460,549 (1.76 MB)

 Non-trainable params: 3,766,816 (14.37 MB)

Epoch 1/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 91s 458ms/step - accuracy: 0.2683 - loss: 2.0882 - val_accuracy: 0.6667 - val_loss: 1.1244
Epoch 2/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.4410 - loss: 1.4730 - val_accuracy: 0.8023 - val_loss: 0.7601
Epoch 3/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.5534 - loss: 1.1486 - val_accuracy: 0.8625 - val_loss: 0.5609
Epoch 4/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.6324 - loss: 0.9759 - val_accuracy: 0.8701 - val_loss: 0.4653
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.6790 - loss: 0.8429 - val_accuracy: 0.8902 - val_loss: 0.3902
Epoch 6/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - accuracy: 0.7131 - loss: 0.7365 - val_accuracy: 0.9026 - val_loss: 0.3514
Epoch 7/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.7559 - loss: 0.6670 - val_accuracy: 0.9064 - val_loss: 0.3099
Epoch 8/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.7734 - loss: 0.5924 - val_accuracy: 0.9207 

Test accuracy: 0.9141221642494202
Saved model successfully!
Class names: ['fist', 'one-finger-up', 'open-palm', 'thumps-up', 'two-fingers']
